### Use the preprocessed data for ML model building. 

In [ ]:
# Setup
import numpy as np 
import pandas as pd 
import os
import pickle

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images/ml_modeling_images")
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

def save_model(model, folder_path="models", file_name="untitled_model.sav"):
    pickle.dump(model, open(os.path.join(folder_path, file_name), 'wb'))

ALZHEIMERS_PATH = "dataset/afterpreprocessing"
alzheimers_dfs = []
file_names = ["Overall_Health", "Mental_Health", "Smoking_and_Alcohol_Use", "Screenings_and_Vaccines", "Nutrition_Physical_Activity_Obesity", "Caregiving", "Cognitive_Decline"]

for i in range(len(file_names)):
    csv_path = os.path.join(ALZHEIMERS_PATH, f'{file_names[i]}.csv')
    df = pd.read_csv(csv_path)
    alzheimers_dfs.append(df)

alzheimers_dfs[0].head()

all_alzheimers_data = pd.read_csv(os.path.join(ALZHEIMERS_PATH, f'all_alzheimers_data.csv'))

all_alzheimers_data.fillna(0, inplace=True)

### Full Feature Set With Decision Tree

In [ ]:
# Training and Visualizing a Decision Tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from graphviz import Source

MODELS_PATH = os.path.join(PROJECT_ROOT_DIR, "models")
DECISION_TREE_PATH = os.path.join(MODELS_PATH, 'decision_tree')

def FullDecisionTreeModel(df):
    tree_clf = DecisionTreeClassifier(max_depth=2, random_state=42)
    for i in range(len(df)):
        input = df[i].drop(['LocationDesc'], axis=1)
        target = df[i]['LocationDesc']
        tree_clf.fit(input, target)
        graph = Source(export_graphviz(tree_clf, out_file=None, feature_names=input.columns, class_names=target, rounded=True, filled=True))
        graph.format = 'png'
        graph.render( IMAGES_PATH + '/decision_tree/' + file_names[i], view=False)
        filename = f'{file_names[i]}_decision_tree_model.sav'
        save_model(tree_clf, DECISION_TREE_PATH, filename)

FullDecisionTreeModel(alzheimers_dfs)

### Full Feature Set With Apriori

In [ ]:
# Apriori https://github.com/ymoch/apyori

from apyori import apriori

def AprioriModel(df):
    for i in range(len(df)):
        records = []
        for j in range(0, df.shape[0]):
            records.append([str(df.values[j, k]) for k in range(0, 33)])
        association_rules = apriori(records, min_support=0.0045, min_confidence=0.2, min_lift=3, min_length=2)
        association_results = list(association_rules)
        print(association_results)
AprioriModel(all_alzheimers_data)

### Full Feature Set with Anomaly Detection

In [98]:
# Anomaly Detection algorthim 

from sklearn.ensemble import IsolationForest

def IsolationForestModel(df):
    for i in range(len(df)):
        input = df[i].drop(['LocationDesc'], axis=1)
        clf = IsolationForest(max_samples=100, random_state=42)
        clf.fit(input)

IsolationForestModel(alzheimers_dfs)

### Full Feature Set With Support Vector Machines

In [97]:
from sklearn.svm import OneClassSVM

OneClassSVM_PATH = os.path.join(MODELS_PATH, 'one_class_svm')

def FullOneClassSVMModel(df):
    input = df.drop(columns=['LocationDesc','AgeGroup'])
    # define and fit outlier detection model
    one_class_svm_clf = OneClassSVM(gamma='auto').fit(input)
    save_model(one_class_svm_clf, OneClassSVM_PATH, 'alzheimers_one_class_svm_model.sav')

FullOneClassSVMModel(all_alzheimers_data)


## Top Feature Selector

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

def top_feature_selector(df, top_num=20):
    input = df.drop(['LocationDesc'], axis=1)
    target = df['LocationDesc']
    bestfeatures = SelectKBest(score_func=chi2, k=3)
    fit = bestfeatures.fit(input,target)
    dfscores = pd.DataFrame(fit.scores_)
    dfcolumns = pd.DataFrame(input.columns)
    featureScores = pd.concat([dfcolumns,dfscores],axis=1)
    featureScores.columns = ['Specs','Score']  #naming the dataframe columns
    print(featureScores.nlargest(top_num,'Score'))  #print 10 best features
    print('----------------------------------------------------')
    return dfcolumns

In [ ]:
# Dropping irrelevant features
all_alzheimers_data = all_alzheimers_data.drop(columns=["Duration of caregiving among older adults", 
                                                        "Intensity of caregiving among older adults", 
                                                        "Talked with health care professional about subjective cognitive decline or memory loss",
                                                        "Expect to provide care for someone in the next two years",
                                                        "Provide care for a friend or family member in past month",
                                                        "Provide care for someone with cognitive impairment within the past month",
                                                        "Self-rated health (fair to poor health)",
                                                        "Self-rated health (good to excellent health)",
                                                        ], axis=1)

In [ ]:
# Separate dataframe by AgeGroup
alzheimers_data_age0 = all_alzheimers_data.loc[all_alzheimers_data['AgeGroup'] == 0].reset_index().drop(['index', 'AgeGroup'], axis=1) # 50-64 y/o
alzheimers_data_age1 = all_alzheimers_data.loc[all_alzheimers_data['AgeGroup'] == 1].reset_index().drop(['index', 'AgeGroup'], axis=1) # 65+ y/o
alzheimers_data_age2 = all_alzheimers_data.loc[all_alzheimers_data['AgeGroup'] == 2].reset_index().drop(['index', 'AgeGroup'], axis=1) # Overall (50+ y/o)

alzheimers_data_age0.head()
alzheimers_data_age1.head()
alzheimers_data_age2.head()

# Get top features for each AgeGroup
age0_features = top_feature_selector(alzheimers_data_age0)
age1_features = top_feature_selector(alzheimers_data_age1)
age2_features = top_feature_selector(alzheimers_data_age2)

alzheimers_data_age0 = alzheimers_data_age0.drop('LocationDesc', axis=1)
alzheimers_data_age1 = alzheimers_data_age1.drop('LocationDesc', axis=1)
alzheimers_data_age2 = alzheimers_data_age2.drop('LocationDesc', axis=1)


In [ ]:
# Keeping the top 20 features
alzheimers_data_age0 = alzheimers_data_age0.drop(columns=age0_features[0][20:].tolist(), axis=1)
alzheimers_data_age1 = alzheimers_data_age1.drop(columns=age1_features[0][20:].tolist(), axis=1)
alzheimers_data_age2 = alzheimers_data_age2.drop(columns=age2_features[0][20:].tolist(), axis=1)

alzheimers_data_age0.head()

### Reduced Feature Set With Decision Tree

In [ ]:
def ReducedDecisionTreeModel(df):
    tree_clf = DecisionTreeClassifier(max_depth=2, random_state=42)
    input =  df.drop(columns='LocationDesc')
    target = df['LocationDesc']
    tree_clf.fit(input, target)
    graph = Source(export_graphviz(tree_clf, out_file=None, feature_names=input.columns, class_names=target, rounded=True, filled=True))
    graph.format = 'png'
    graph.render( IMAGES_PATH + '/decision_tree/all_alzheimers_data', view=False)
    filename = 'all_alzheimers_data_decision_tree_model.sav'
    save_model(tree_clf, DECISION_TREE_PATH, filename)

ReducedDecisionTreeModel(all_alzheimers_data)

### Reduced Feature Set With K-Means

In [80]:
from sklearn.cluster import KMeans

KMEANS_PATH = os.path.join(MODELS_PATH, 'kmeans')

kmeans_clf = KMeans(n_clusters=2, random_state=0, n_init="auto").fit(alzheimers_data_age0)

save_model(kmeans_clf, KMEANS_PATH, 'alzheimers_kmeans_model.sav')


TypeError: '<=' not supported between instances of 'str' and 'int'

### Reduced Feature Set With Support Vector Machines

In [103]:
from sklearn.svm import OneClassSVM

def ReducedOneClassSVMModel(df):
    input = df
    # define and fit outlier detection model
    one_class_svm_clf = OneClassSVM(gamma='auto').fit(input)
    save_model(one_class_svm_clf, OneClassSVM_PATH, 'alzheimers_one_class_svm_model.sav')

ReducedOneClassSVMModel(alzheimers_data_age0)


## Load Models

In [105]:

loaded_decision_tree_model = pickle.load(open(os.path.join(DECISION_TREE_PATH, "Overall_Health_decision_tree_model.sav"), 'rb'))
loaded_kmeans_model = pickle.load(open(os.path.join(KMEANS_PATH, "alzheimers_kmeans_model.sav"), 'rb'))
loaded_one_class_svm_model = pickle.load(open(os.path.join(OneClassSVM_PATH, "alzheimers_one_class_svm_model.sav"), 'rb'))